In [2]:
import numpy as np
import re
import pandas as pd
from tqdm.notebook import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from typing import List, Optional
import enum

from langchain_community.llms import Ollama
from langchain.output_parsers.regex_dict import RegexDictParser
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage, SystemMessage, ChatMessage
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from pydantic import BaseModel, Field, validator, create_model
from openai import AsyncOpenAI, OpenAI
#import asyncio
import os
from typing import Tuple, List, Optional

import requests

from pydantic import BaseModel, ValidationInfo, model_validator

import json

import itertools
from copy import deepcopy
from tqdm.notebook import tqdm, trange
from sklearn.cluster import KMeans

import umap.umap_ as umap
#import umap
import hdbscan

from src.bubble import *
from src.models import *
from src.utilities import *

Retrieved company Darty : 1707313014508x102198350946437700
Retrieved project Expérience utilisateur de Darty : 1710335239022x546888753774592000


In [3]:
types_df = get("Type", constraints=[])
categories_df = get("Category")
subcategories_df = get("SubCategory")
aspects_df = get("Aspect")

In [4]:
aspects_df.to_csv("aspects_df.csv")

In [14]:
aspects_df

,Company,Project,Rating,SubCategory,Associated_feedback,Date,Category,_id,Explanation
0,1707313014508x102198350946437700,1710335239022x546888753774592000,5,1710350416999x528701117282035500,1710335410193x146397114086602720,2024-02-07 00:00:00+00:00,1710350416439x770681541783434100,1710359842200x821681263546503200,NaN
1,1707313014508x102198350946437700,1710335239022x546888753774592000,5,1710350410151x820615919729764900,1710335410193x146397114086602720,2024-02-07 00:00:00+00:00,1710350409745x445953751757016450,1710359842221x274758448864749860,Le clavier Magic Keyboard avec Touch ID et pav...
2,1707313014508x102198350946437700,1710335239022x546888753774592000,5,1710350421272x199685951705496740,1710335410195x175970366374120320,2024-01-31 00:00:00+00:00,1710350419545x536227079121094340,1710359842248x367738921884239740,Installation professionnelle incluant la repri...
3,1707313014508x102198350946437700,1710335239022x546888753774592000,5,1710350410151x820615919729764900,1710335410202x536776314861663740,2024-01-05 00:00:00+00:00,1710350409745x445953751757016450,1710359842258x625624914090439300,La machine à laver Bosch offre une simplicité ...
4,1707313014508x102198350946437700,1710335239022x546888753774592000,5,1710350419870x657684400109240000,1710335410202x536776314861663740,2024-01-05 00:00:00+00:00,1710350419545x536227079121094340,1710359842268x848738395758196600,Livraison impeccable et très soignée.
...,...,...,...,...,...,...,...,...,...
10255,1707313014508x102198350946437700,1710335239022x546888753774592000,1,1710350426048x584251813766510200,1710335488652x833235652633474300,2011-12-30 00:00:00+00:00,1710350422859x678271645638851300,1710360011763x508529931406086460,Difficultés rencontrées pour obtenir le rembou...
10256,1707313014508x102198350946437700,1710335239022x546888753774592000,5,1710350415153x853649571631085700,1710335488654x498578120665668540,2011-12-01 00:00:00+00:00,1710350412899x638565534548052100,1710360011775x865663617847532300,NaN
10257,1707313014508x102198350946437700,1710335239022x546888753774592000,5,1710350428098x591651016314862500,1710335488655x206660558179535330,2011-09-15 00:00:00+00:00,1710350426602x488976415178478340,1710360011822x943753414489857400,Service assistance très aimable et professionn...
10258,1707313014508x102198350946437700,1710335239022x546888753774592000,1,1710350424753x747645709223324400,1710335488659x219695565294586020,2011-09-04 00:00:00+00:00,1710350422859x678271645638851300,1710360011830x146365060294136420,"Service après-vente jugé incompétent, frais su..."


In [5]:
ID_CATEG_NONE = categories_df[categories_df["Name"].isna()].iloc[0]["_id"]
REAL_CATEGORIES = categories_df[categories_df["_id"] != ID_CATEG_NONE]
ID_SUBCATEGS_NONE = list(subcategories_df[subcategories_df["Name"].isna()]["_id"])


In [23]:
import pandas as pd


def analyze_feedback_by_period(df, company, category, subcategory, period):
  """Analyzes customer feedback by company, category, subcategory, and period.

  Args:
      df: A pandas DataFrame containing the customer feedback data.
      company: The company to filter by.
      category: The category to filter by.
      subcategory: The subcategory to filter by.
      period: The desired period for grouping the data (e.g., "D" for day, "W" for week, "M" for month, "Y" for year).

  Returns:
      A pandas DataFrame with the following columns:
          * Date: The index of the DataFrame, containing the dates formatted according to the period, 
                  starting from the minimum date of the entire DataFrame.
          * count: The number of customer feedback entries for each period (0 if no entries).
          * ... (similar columns for rating counts, mean, and standard deviation)
          
      The min and std columns will have None values for periods with no entries.
  """

  # Get the minimum date from the entire DataFrame
  min_date = df["Date"].min()

  # Filter the DataFrame by the specified criteria

  filter = (df["Company"] == company)
  if category != ID_CATEG_NONE:
    filter &= (df["Category"] == category)
    if subcategory not in ID_SUBCATEGS_NONE:
      filter &= (df["SubCategory"] == subcategory)

  filtered_df = df[filter]

  # Handle cases where no data matches the filter
  if filtered_df.empty:
    res_df = pd.DataFrame({
        "Date": pd.date_range(start=min_date, periods=0, freq=period)})
    res_df["count"]= 0
    res_df["rating_1"]= 0
    res_df["rating_2"]= 0
    res_df["rating_3"]= 0
    res_df["rating_4"]= 0
    res_df["rating_5"]= 0
    res_df["min"]= 0
    res_df["max"]= 0
    res_df["mean"]= 0
    res_df["std"]= 0
    res_df["median"]= 0
    res_df["q1"]= 0
    res_df["q3"]= 0
    res_df["Date Display"] = res_df["Date"].apply(
      lambda x: format_date(x, period)
  )
    return res_df
  
  # Convert the "Date" column to datetime format
  filtered_df["Date"] = pd.to_datetime(filtered_df["Date"])

  # Define date format function based on period
  def format_date(date, period):
    if period == "D":
      return date.strftime("%d/%m/%Y")
    elif period == "W":
      return date.strftime("%U-%Y")  # Week format with year, e.g., 01-2024
    elif period == "M":
      return date.strftime("%m/%Y")
    else:
      return date.strftime("%Y")

  # Resample the data by the specified period, aggregating on various columns
  grouped_data = (
      filtered_df.set_index("Date")
      .resample(period)
      .agg(
          count=("Rating", "count"),
          rating_1=("Rating", lambda x: x[x == 1].count()),
          rating_2=("Rating", lambda x: x[x == 2].count()),
          rating_3=("Rating", lambda x: x[x == 3].count()),
          rating_4=("Rating", lambda x: x[x == 4].count()),
          rating_5=("Rating", lambda x: x[x == 5].count()),
          mean=("Rating", "mean"),
          std=("Rating", "std"),
          median=("Rating", "median"),
          min=("Rating", "min"),
          max=("Rating", "max"),
          q1=("Rating", lambda x: x.quantile(0.25)),
          q3=("Rating", lambda x: x.quantile(0.75)),
          #explanation=("Explanations", lambda x: list(x)),
      )
      .reset_index()
  )

  
  # Set the minimum date from entire DataFrame as the starting index
  grouped_data.set_index("Date", inplace=True)
  # Create a date range with appropriate length based on min and max dates of filtered data
  date_range = pd.date_range(
      start=min_date, end=filtered_df["Date"].max(), freq=period
  )

  # Reindex to fill missing periods with 0 and None for mean/std
  grouped_data = grouped_data.reindex(date_range).fillna(0)

  # Fill missing values with appropriate values
  grouped_data.loc[:, "count"] = grouped_data["count"].fillna(0)
  grouped_data.loc[grouped_data["count"] == 0, ["mean", "std", "median", "min", "max", "q1", "q3"]] = None

  grouped_data = grouped_data.rename_axis('Date').reset_index()

  # Apply date formatting based on period
  grouped_data["Date Display"] = grouped_data["Date"].apply(
      lambda x: format_date(x, period)
  )
  return grouped_data


In [24]:
analyze_feedback_by_period(aspects_df, COMPANY_ID, categories_df.iloc[0]["_id"], subcategories_df.iloc[0]["_id"], "M")

/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_80922/2387198549.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["Date"] = pd.to_datetime(filtered_df["Date"])
/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_80922/2387198549.py:76: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(period)


ValueError: func must be a callable if args or kwargs are supplied

In [7]:
def to_list(s):
    return [0 if pd.isna(x) else x for x in s]

def send_aspects(df, company, category, subcategory, period, grouped_by=None):
    d = {
        "Company": company,
        "Project": PROJECT_ID,
        "Category": category,
        "SubCategory": subcategory,
        "Period": period,
        "Grouped by": grouped_by,
        "Counts": to_list(df["count"]),
        "Counts of 1s": to_list(df["rating_1"]),
        "Counts of 2s": to_list(df["rating_2"]),
        "Counts of 3s": to_list(df["rating_3"]),
        "Counts of 4s": to_list(df["rating_4"]),
        "Counts of 5s": to_list(df["rating_5"]),
        "Dates": to_list(df["Date"].apply(lambda x:pd.to_datetime(x).strftime("%m/%d/%Y"))), #
        "Dates Display": to_list(df["Date Display"]),
        "Max Ratings": to_list(df["min"]),
        "Mean Ratings": to_list(df["mean"]),
        "Std Ratings": to_list(df["std"]),
        "Med Ratings": to_list(df["median"]),
        "Min Ratings": to_list(df["max"]),
        "Q1 Rating": to_list(df["q1"]),
        "Q3 Rating": to_list(df["q3"]),
        }  

    bubble_id = bubble_client.create("Aspect Evol",d)
    return bubble_id

In [8]:
aspects_df

,Company,Project,Rating,SubCategory,Associated_feedback,Date,Category,_id,Explanation
0,1707313014508x102198350946437700,1710335239022x546888753774592000,5,1710350416999x528701117282035500,1710335410193x146397114086602720,2024-02-07 00:00:00+00:00,1710350416439x770681541783434100,1710359842200x821681263546503200,NaN
1,1707313014508x102198350946437700,1710335239022x546888753774592000,5,1710350410151x820615919729764900,1710335410193x146397114086602720,2024-02-07 00:00:00+00:00,1710350409745x445953751757016450,1710359842221x274758448864749860,Le clavier Magic Keyboard avec Touch ID et pav...
2,1707313014508x102198350946437700,1710335239022x546888753774592000,5,1710350421272x199685951705496740,1710335410195x175970366374120320,2024-01-31 00:00:00+00:00,1710350419545x536227079121094340,1710359842248x367738921884239740,Installation professionnelle incluant la repri...
3,1707313014508x102198350946437700,1710335239022x546888753774592000,5,1710350410151x820615919729764900,1710335410202x536776314861663740,2024-01-05 00:00:00+00:00,1710350409745x445953751757016450,1710359842258x625624914090439300,La machine à laver Bosch offre une simplicité ...
4,1707313014508x102198350946437700,1710335239022x546888753774592000,5,1710350419870x657684400109240000,1710335410202x536776314861663740,2024-01-05 00:00:00+00:00,1710350419545x536227079121094340,1710359842268x848738395758196600,Livraison impeccable et très soignée.
...,...,...,...,...,...,...,...,...,...
10255,1707313014508x102198350946437700,1710335239022x546888753774592000,1,1710350426048x584251813766510200,1710335488652x833235652633474300,2011-12-30 00:00:00+00:00,1710350422859x678271645638851300,1710360011763x508529931406086460,Difficultés rencontrées pour obtenir le rembou...
10256,1707313014508x102198350946437700,1710335239022x546888753774592000,5,1710350415153x853649571631085700,1710335488654x498578120665668540,2011-12-01 00:00:00+00:00,1710350412899x638565534548052100,1710360011775x865663617847532300,NaN
10257,1707313014508x102198350946437700,1710335239022x546888753774592000,5,1710350428098x591651016314862500,1710335488655x206660558179535330,2011-09-15 00:00:00+00:00,1710350426602x488976415178478340,1710360011822x943753414489857400,Service assistance très aimable et professionn...
10258,1707313014508x102198350946437700,1710335239022x546888753774592000,1,1710350424753x747645709223324400,1710335488659x219695565294586020,2011-09-04 00:00:00+00:00,1710350422859x678271645638851300,1710360011830x146365060294136420,"Service après-vente jugé incompétent, frais su..."


In [9]:
subcategories_df

,Company,Name,Project,Category,_id
0,1707313014508x102198350946437700,Fonctionnalité,1710335239022x546888753774592000,1710350409745x445953751757016450,1710350410151x820615919729764900
1,1707313014508x102198350946437700,Durabilité,1710335239022x546888753774592000,1710350409745x445953751757016450,1710350410504x667859650200292500
2,1707313014508x102198350946437700,Conformité,1710335239022x546888753774592000,1710350409745x445953751757016450,1710350411096x295295631386005300
3,1707313014508x102198350946437700,Performance,1710335239022x546888753774592000,1710350409745x445953751757016450,1710350411420x525018832955180160
4,1707313014508x102198350946437700,Design,1710335239022x546888753774592000,1710350409745x445953751757016450,1710350411818x301326900426685440
5,1707313014508x102198350946437700,Rapport qualité-prix,1710335239022x546888753774592000,1710350409745x445953751757016450,1710350412263x819591333567755400
6,1707313014508x102198350946437700,NaN,1710335239022x546888753774592000,1710350409745x445953751757016450,1710350412594x226453784069726050
7,1707313014508x102198350946437700,Amabilité,1710335239022x546888753774592000,1710350412899x638565534548052100,1710350413272x150069318891535200
8,1707313014508x102198350946437700,Efficacité,1710335239022x546888753774592000,1710350412899x638565534548052100,1710350413615x727017993028421800
9,1707313014508x102198350946437700,Disponibilité,1710335239022x546888753774592000,1710350412899x638565534548052100,1710350414242x828234744357746300


In [13]:
for category in tqdm(categories_df["_id"]):
    subcategories = subcategories_df[subcategories_df["Category"]==category]["_id"]

    for subcategory in subcategories:
        for period in ["M", "Y"]: #["D","W", "M", "Y"]:
            df = analyze_feedback_by_period(aspects_df, COMPANY_ID, category, subcategory, period)
            send_aspects(df, COMPANY_ID, category, subcategory, period)

  0%|          | 0/7 [00:00<?, ?it/s]

/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_80922/2947707909.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["Date"] = pd.to_datetime(filtered_df["Date"])
/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_80922/2947707909.py:76: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(period)
/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_80922/2947707909.py:99: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  date_range = pd.date_range(
/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_80922/2947707909.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [ ]:
#df[[f"Count of {i}s" for i in range(1,6)]].plot(kind='bar', stacked=True)
